In [27]:
import os
import time
import numpy as np
import pandas as pd

# --- GDT imports ---
from gdt.missions.fermi.gbm.tte import GbmTte
from gdt.missions.fermi.gbm.phaii import GbmPhaii
from gdt.missions.fermi.gbm.collection import GbmDetectorCollection
from gdt.core.binning.unbinned import bin_by_time
from gdt.core.background.fitter import BackgroundFitter
from gdt.core.background.binned import Polynomial
from gdt.missions.fermi.gbm.response import GbmRsp2 as RSP
from gdt.core.spectra.functions import Band
from gdt.core.spectra.fitting import SpectralFitterCstat

# ============================================================
# === CONFIGURACIÓN DE ARCHIVOS ===
# ============================================================

fitn7 = os.path.join('data', 'glg_tte_n7_bn090926181_v00.fit')
tte7 = GbmTte.open(fitn7)

test_pha_n7 = os.path.join('data', 'glg_cspec_n7_bn090926181_v00.pha')
test_pha_n3 = os.path.join('data', 'glg_cspec_n3_bn090926181_v00.pha')
test_pha_b0 = os.path.join('data', 'glg_cspec_b0_bn090926181_v00.pha')
n7 = GbmPhaii.open(test_pha_n7)
n3 = GbmPhaii.open(test_pha_n3)
b0 = GbmPhaii.open(test_pha_b0)

cspecs = GbmDetectorCollection.from_list([n7, n3, b0])
phaii7 = tte7.to_phaii(bin_by_time, 1.024, time_ref=0.0)

erange_nai = (8.0, 900.0)
erange_bgo = (325., 25000.)

# ============================================================
# === FUNCIÓN AUXILIAR: CÁLCULO DE T90 ===
# ============================================================

def get_T90(phaii):
    """
    Compute T5, T95, and T90 for a GBM light curve
    """
    times = phaii.data.time_centroids
    counts = phaii.data.counts.sum(axis=1)

    cumulative = np.cumsum(counts)
    cumulative = (cumulative - cumulative.min()) / (cumulative.max() - cumulative.min())

    T5  = np.interp(0.05, cumulative, times)
    T95 = np.interp(0.95, cumulative, times)
    T90 = T95 - T5
    return float(T5), float(T95), float(T90)

# ============================================================
# === PASO 1. AJUSTE DE BACKGROUND ===
# ============================================================

T5, T95, T90 = get_T90(phaii7)
bkgd_range = [(T5 - 50, T5 - 10), (T95 + 10, T95 + 50)]

backfitters = [BackgroundFitter.from_phaii(cspec, Polynomial, time_ranges=bkgd_range) for cspec in cspecs]
backfitters = GbmDetectorCollection.from_list(backfitters, dets=cspecs.detector())
backfitters.fit(order=1)

bkgds = backfitters.interpolate_bins(cspecs.data()[0].tstart, cspecs.data()[0].tstop)
bkgds = GbmDetectorCollection.from_list(bkgds, dets=cspecs.detector())

# ============================================================
# === PASO 2. DETECCIÓN AUTOMÁTICA DEL BURST ===
# ============================================================

counts = phaii7.data.counts.sum(axis=1)
times = phaii7.time_range[0] + np.arange(len(counts)) * 1.0

bkg_level = np.percentile(counts, 10)
threshold = bkg_level + 3 * np.std(counts[counts < np.percentile(counts, 50)])

mask = counts > threshold
if not np.any(mask):
    raise RuntimeError("No se detectó ningún destello")

burst_start = times[np.argmax(mask)]
burst_end = times[len(mask) - np.argmax(mask[::-1]) - 1] + 1.0

print(f"💥 Burst detectado entre {burst_start:.2f} s y {burst_end:.2f} s")

# ============================================================
# === PASO 3. DEFINICIÓN DE INTERVALOS ===
# ============================================================

time_edges = np.arange(burst_start, burst_end, 1.0)
time_ranges = [(t, t + 1.0) for t in time_edges if t + 1.0 <= burst_end]

# ============================================================
# === PASO 4. CONFIGURACIÓN DE RESPUESTAS Y MODELO ===
# ============================================================

rsp_dir_n7 = os.path.join('data', 'glg_cspec_n7_bn090926181_v00.rsp2')
rsp_dir_n3 = os.path.join('data', 'glg_cspec_n3_bn090926181_v00.rsp2')
rsp_dir_b0 = os.path.join('data', 'glg_cspec_b0_bn090926181_v00.rsp2')

rsp1 = RSP.open(rsp_dir_n7)
rsp2 = RSP.open(rsp_dir_n3)
rsp3 = RSP.open(rsp_dir_b0)
rsps = GbmDetectorCollection.from_list([rsp1, rsp2, rsp3])

model = Band()
model.min_values = [1e-10, 0.01, -1.9, -10.0, 0.01]
model.max_values = [np.inf, 5e4, 1.5, -2.05, np.inf]

model.default_values = [0.01, 500.0, -0.5, -2.3, 100.0]

# Fijar beta (posición 3)
model.fix = [False, False, False, True, False]

results = []

# ============================================================
# === OPTIMIZACIÓN 1: Precalcular respuestas interpoladas ===
# ============================================================

rsps_interpoladas = []
for (t0, t1) in time_ranges:
    tcent = (t0 + t1) / 2
    rsps_interpoladas.append([rsp.interpolate(tcent) for rsp in rsps])

# ============================================================
# === PASO 5. LOOP DE AJUSTES ESPECTRALES ===
# ============================================================

for i, (t0, t1) in enumerate(time_ranges):

    phas = cspecs.to_pha(time_ranges=[(t0, t1)], nai_kwargs={'energy_range':erange_nai}, bgo_kwargs={'energy_range':erange_bgo})

    rsps_interp = rsps_interpoladas[i]  # Usar precálculo

    fitter = SpectralFitterCstat(phas, bkgds.to_list(), rsps_interp, method='TNC')

    try:
        start_time = time.time()

        # OPTIMIZACIÓN 2: Reducir número máximo de iteraciones
        try:
            fitter.fit(model, options={'maxiter': 500})
        except Exception as e:
            print(f"⚠️ Error interno en fit: {e}")
            continue

        # Timeout
        if time.time() - start_time > 10:
            print("⏰ Timeout del fit — se detiene este intervalo.")
            continue

        # Validar convergencia
        if not np.isfinite(fitter.statistic):
            print("⚠️ Fit no válido (stat no finita)")
            continue
        if fitter.covariance is None or np.isnan(fitter.covariance).any():
            print("⚠️ Covarianza inválida.")
            continue

        # === Calcular errores relativos (solo alpha, beta y Epeak) ===
        errs = np.array(fitter.asymmetric_errors(cl=0.9), dtype=float)
        params = np.array(fitter.parameters)

        if len(errs) != len(params):
            print(f"⚠️ Inconsistencia: {len(errs)} errores para {len(params)} parámetros.")
            continue

        #rel_errs = np.mean(np.abs(errs) / np.abs(params[:, None]), axis=1)
        eps = 1e-10
        rel_errs = np.mean(
            np.abs(errs) / np.maximum(np.abs(params[:, None]), eps),
            axis=1
        )

        # Índices del modelo Band → [A, Epeak, alpha, beta, Epiv]
        indices_eval = [1, 2]
        rel_errs_eval = rel_errs[indices_eval]
        max_rel = np.max(rel_errs_eval)

        # === Guardar resultados válidos ===
        if max_rel < 0.2:
            res = {
                "t_start": t0,
                "t_stop": t1,
                "Cstat": fitter.statistic,
                "dof": fitter.dof,
            }

            # Parámetros y errores
            for name, val in zip(model.param_list, fitter.parameters):
                res[name[0]] = float(val)

            for i, (name, val) in enumerate(zip(model.param_list, fitter.parameters)):
                err_low, err_high = errs[i]
                rel_err_mean = rel_errs[i] * 100
                res[f"{name[0]}_err_low"] = float(err_low)
                res[f"{name[0]}_err_high"] = float(err_high)
                res[f"{name[0]}_err_rel(%)"] = float(rel_err_mean)

            results.append(res)
            print(f"{t0:.3f}–{t1:.3f} s: ✅ Fit guardado.")
        else:
            print(f"{t0:.3f}–{t1:.3f} s: ❌ Errores grandes ({max_rel*100:.1f}%)")

    except Exception as e:
        print(f"Error en {t0}-{t1}: {e}")
        continue

# ============================================================
# === PASO 6. GUARDAR RESULTADOS ===
# ============================================================

if results:
    df = pd.DataFrame(results)
    df.to_csv("valid_fits.csv", index=False)
    print(f"✅ {len(df)} valid fits saved to 'valid_fits.csv'")
else:
    print("⚠️ No valid fits found.")


💥 Burst detectado entre -1.65 s y 30.35 s


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


-1.648–-0.648 s: ❌ Errores grandes (79.3%)


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


-0.648–0.352 s: ❌ Errores grandes (33.9%)


...

/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


27.352–28.352 s: ❌ Errores grandes (88.3%)


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


28.352–29.352 s: ❌ Errores grandes (108.5%)
29.352–30.352 s: ❌ Errores grandes (78.5%)
✅ 17 valid fits saved to 'valid_fits.csv'


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar resultados
df = pd.read_csv("valid_fits.csv")

# Calcular tiempo central de cada bin
df["t_center"] = 0.5 * (df["t_start"] + df["t_stop"])

# === Graficar cada parámetro ===
params = ["Epeak", "alpha", "beta"]

for p in params:
    if p not in df.columns:
        print(f"⚠️ {p} no está en el CSV, se omite.")
        continue

    err_low = df[f"{p}_err_low"].abs()
    err_high = df[f"{p}_err_high"].abs()

    plt.figure(figsize=(6, 4))
    plt.errorbar(
        df["t_center"],
        df[p],
        yerr=[err_low, err_high],
        fmt="o",
        ecolor="gray",
        elinewidth=1,
        capsize=3,
        ms=4,
        color="tab:blue"
    )
    plt.xlabel("Tiempo (s)")
    plt.ylabel(p)
    plt.title(f"Evolución temporal de {p}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [22]:
import os
import time
import numpy as np
import pandas as pd

# --- GDT imports ---
from gdt.missions.fermi.gbm.tte import GbmTte
from gdt.missions.fermi.gbm.phaii import GbmPhaii
from gdt.missions.fermi.gbm.collection import GbmDetectorCollection
from gdt.core.binning.unbinned import bin_by_time
from gdt.core.background.fitter import BackgroundFitter
from gdt.core.background.binned import Polynomial
from gdt.missions.fermi.gbm.response import GbmRsp2 as RSP
from gdt.core.spectra.functions import BlackBody, Comptonized, PowerLaw
from gdt.core.spectra.fitting import SpectralFitterCstat

# ============================================================
# === CONFIGURACIÓN DE ARCHIVOS ===
# ============================================================

fitn7 = os.path.join('data', 'glg_tte_n7_bn090926181_v00.fit')
tte7 = GbmTte.open(fitn7)

test_pha_n7 = os.path.join('data', 'glg_cspec_n7_bn090926181_v00.pha')
test_pha_n3 = os.path.join('data', 'glg_cspec_n3_bn090926181_v00.pha')
test_pha_b0 = os.path.join('data', 'glg_cspec_b0_bn090926181_v00.pha')
n7 = GbmPhaii.open(test_pha_n7)
n3 = GbmPhaii.open(test_pha_n3)
b0 = GbmPhaii.open(test_pha_b0)

cspecs = GbmDetectorCollection.from_list([n7, n3, b0])
phaii7 = tte7.to_phaii(bin_by_time, 1.024, time_ref=0.0)

erange_nai = (8.0, 900.0)
erange_bgo = (325., 25000.)

# ============================================================
# === FUNCIÓN AUXILIAR: CÁLCULO DE T90 ===
# ============================================================

def get_T90(phaii):
    """
    Compute T5, T95, and T90 for a GBM light curve
    """
    times = phaii.data.time_centroids
    counts = phaii.data.counts.sum(axis=1)

    cumulative = np.cumsum(counts)
    cumulative = (cumulative - cumulative.min()) / (cumulative.max() - cumulative.min())

    T5  = np.interp(0.05, cumulative, times)
    T95 = np.interp(0.95, cumulative, times)
    T90 = T95 - T5
    return float(T5), float(T95), float(T90)

# ============================================================
# === PASO 1. AJUSTE DE BACKGROUND ===
# ============================================================

T5, T95, T90 = get_T90(phaii7)
bkgd_range = [(T5 - 50, T5 - 10), (T95 + 10, T95 + 50)]

backfitters = [BackgroundFitter.from_phaii(cspec, Polynomial, time_ranges=bkgd_range) for cspec in cspecs]
backfitters = GbmDetectorCollection.from_list(backfitters, dets=cspecs.detector())
backfitters.fit(order=1)

bkgds = backfitters.interpolate_bins(cspecs.data()[0].tstart, cspecs.data()[0].tstop)
bkgds = GbmDetectorCollection.from_list(bkgds, dets=cspecs.detector())

# ============================================================
# === PASO 2. DETECCIÓN AUTOMÁTICA DEL BURST ===
# ============================================================

counts = phaii7.data.counts.sum(axis=1)
times = phaii7.time_range[0] + np.arange(len(counts)) * 1.0

bkg_level = np.percentile(counts, 10)
threshold = bkg_level + 3 * np.std(counts[counts < np.percentile(counts, 50)])

mask = counts > threshold
if not np.any(mask):
    raise RuntimeError("No se detectó ningún destello")

burst_start = times[np.argmax(mask)]
burst_end = times[len(mask) - np.argmax(mask[::-1]) - 1] + 1.0

print(f"💥 Burst detectado entre {burst_start:.2f} s y {burst_end:.2f} s")

# ============================================================
# === PASO 3. DEFINICIÓN DE INTERVALOS ===
# ============================================================

time_edges = np.arange(burst_start, burst_end, 1.0)
time_ranges = [(t, t + 1.0) for t in time_edges if t + 1.0 <= burst_end]

# ============================================================
# === PASO 4. CONFIGURACIÓN DE RESPUESTAS Y MODELO ===
# ============================================================

rsp_dir_n7 = os.path.join('data', 'glg_cspec_n7_bn090926181_v00.rsp2')
rsp_dir_n3 = os.path.join('data', 'glg_cspec_n3_bn090926181_v00.rsp2')
rsp_dir_b0 = os.path.join('data', 'glg_cspec_b0_bn090926181_v00.rsp2')

rsp1 = RSP.open(rsp_dir_n7)
rsp2 = RSP.open(rsp_dir_n3)
rsp3 = RSP.open(rsp_dir_b0)
rsps = GbmDetectorCollection.from_list([rsp1, rsp2, rsp3])

model = Comptonized() + PowerLaw() + BlackBody()
#model.free = [True] * model.nparams  # fuerza que todos los parámetros sean libres
model.default_values = [0.01, 300.0, -0.7, 100.0, 0.1, -1.5, 100.0, 0.01, 30.0]
model.free = [True, True, False, True, True, False, True, True, True]

results = []

# ============================================================
# === OPTIMIZACIÓN 1: Precalcular respuestas interpoladas ===
# ============================================================

rsps_interpoladas = []
for (t0, t1) in time_ranges:
    tcent = (t0 + t1) / 2
    rsps_interpoladas.append([rsp.interpolate(tcent) for rsp in rsps])

# ============================================================
# === PASO 5. LOOP DE AJUSTES ESPECTRALES ===
# ============================================================

for i, (t0, t1) in enumerate(time_ranges):

    phas = cspecs.to_pha(time_ranges=[(t0, t1)], nai_kwargs={'energy_range':erange_nai}, bgo_kwargs={'energy_range':erange_bgo})

    rsps_interp = rsps_interpoladas[i]  # Usar precálculo

    fitter = SpectralFitterCstat(phas, bkgds.to_list(), rsps_interp, method='TNC')

    try:
        start_time = time.time()

        # OPTIMIZACIÓN 2: Reducir número máximo de iteraciones
        try:
            fitter.fit(model, options={'maxiter': 500})
        except Exception as e:
            print(f"⚠️ Error interno en fit: {e}")
            continue

        # Timeout
        if time.time() - start_time > 10:
            print("⏰ Timeout del fit — se detiene este intervalo.")
            continue

        # Validar convergencia
        if not np.isfinite(fitter.statistic):
            print("⚠️ Fit no válido (stat no finita)")
            continue
        if fitter.covariance is None or np.isnan(fitter.covariance).any():
            print("⚠️ Covarianza inválida.")
            continue

        # === Calcular errores relativos ===
        errs = np.array(fitter.asymmetric_errors(cl=0.9), dtype=float)
        params = np.array(fitter.parameters)

        if len(errs) != len(params):
            print(f"⚠️ Inconsistencia: {len(errs)} errores para {len(params)} parámetros.")
            continue

        rel_errs = np.mean(np.abs(errs) / np.abs(params[:, None]), axis=1)

        # Índices del modelo BlackBody → [Ac, EpeakC, indexC, EpivC, Ap, indexP, EpivP, Ab, KT]
        indices_eval = [1,6]
        rel_errs_eval = rel_errs[indices_eval]
        max_rel = np.max(rel_errs_eval)

        # === Guardar resultados válidos ===
        if max_rel < 0.2:
            res = {
                "t_start": t0,
                "t_stop": t1,
                "Cstat": fitter.statistic,
                "dof": fitter.dof,
            }

            # Parámetros y errores
            for name, val in zip(model.param_list, fitter.parameters):
                res[name[0]] = float(val)

            for i, (name, val) in enumerate(zip(model.param_list, fitter.parameters)):
                err_low, err_high = errs[i]
                rel_err_mean = rel_errs[i] * 100
                res[f"{name[0]}_err_low"] = float(err_low)
                res[f"{name[0]}_err_high"] = float(err_high)
                res[f"{name[0]}_err_rel(%)"] = float(rel_err_mean)

            results.append(res)
            print(f"{t0:.3f}–{t1:.3f} s: ✅ Fit guardado.")
        else:
            print(f"{t0:.3f}–{t1:.3f} s: ❌ Errores grandes ({max_rel*100:.1f}%)")

    except Exception as e:
        print(f"Error en {t0}-{t1}: {e}")
        continue

# ============================================================
# === PASO 6. GUARDAR RESULTADOS ===
# ============================================================

if results:
    df = pd.DataFrame(results)
    df.to_csv("bb_fits.csv", index=False)
    print(f"✅ {len(df)} valid fits saved to 'bb_fits.csv'")
else:
    print("⚠️ No valid fits found.")


💥 Burst detectado entre -1.65 s y 30.35 s


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/functions.py:926: RuntimeWarning: overflow encountered in exp
  fxn = A * x ** 2 / (np.exp(x / kT) - 1.0)


-1.648–-0.648 s: ❌ Errores grandes (9948.4%)


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/functions.py:926: RuntimeWarning: overflow encountered in exp
  fxn = A * x ** 2 / (np.exp(x / kT) - 1.0)


-0.648–0.352 s: ❌ Errores grandes (2053.5%)


...

/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/functions.py:926: RuntimeWarning: overflow encountered in exp
  fxn = A * x ** 2 / (np.exp(x / kT) - 1.0)
/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:368: RuntimeWarning: overflow encountered in scalar multiply
  maxval = 2.0 * param
/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")


28.352–29.352 s: ❌ Errores grandes (inf%)


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/functions.py:926: RuntimeWarning: overflow encountered in exp
  fxn = A * x ** 2 / (np.exp(x / kT) - 1.0)


29.352–30.352 s: ❌ Errores grandes (inf%)
✅ 7 valid fits saved to 'bb_fits.csv'


/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:368: RuntimeWarning: overflow encountered in scalar multiply
  maxval = 2.0 * param
/home/reegh/Documentos/Astro/GRBs-art/.gdt/lib/python3.11/site-packages/gdt/core/spectra/fitting.py:381: UserWarning: Parameter exists at its upper bound
  warnings.warn("Parameter exists at its upper bound")
